Byte-Pair Encoding (BPE) was initially developed as an algorithm to compress texts and it is used in GPT, GPT-2, RoBERTa, BART, and DeBERTa

BPE training starts by **computing the unique set of words** used in the corpus (after the normalization and pre-tokenization steps are completed), then building the vocabulary by taking all the symbols used to write those words


In [2]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

1. use the gpt2 tokenizer for the pre-tokenization

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2") # we’re using a GPT2 tokenizer, as pre-tokenization 
                                                  #involves splitting on whitespace and punctuation

/home/qxy699/hugging_face/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.  compute the frequencies of **each word** in the corpus as we do the pre-tokenization

In [13]:
word_freqs = dict()

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    words = [word for word, offsets in words_with_offsets]
    for word in words:
        if word not in word_freqs.keys():
            word_freqs[word] = 1
        else:
            word_freqs[word] += 1

word_freqs


{'This': 3,
 'Ġis': 2,
 'Ġthe': 1,
 'ĠHugging': 1,
 'ĠFace': 1,
 'ĠCourse': 1,
 '.': 4,
 'Ġchapter': 1,
 'Ġabout': 1,
 'Ġtokenization': 1,
 'Ġsection': 1,
 'Ġshows': 1,
 'Ġseveral': 1,
 'Ġtokenizer': 1,
 'Ġalgorithms': 1,
 'Hopefully': 1,
 ',': 1,
 'Ġyou': 1,
 'Ġwill': 1,
 'Ġbe': 1,
 'Ġable': 1,
 'Ġto': 1,
 'Ġunderstand': 1,
 'Ġhow': 1,
 'Ġthey': 1,
 'Ġare': 1,
 'Ġtrained': 1,
 'Ġand': 1,
 'Ġgenerate': 1,
 'Ġtokens': 1}

3. compute the base vocabulary(those ones created in the tokenizer), formed by all the characters used in the corpus

In [19]:
alphabet = []

for keys in word_freqs.keys():
    char = [chrs for chrs in keys]
    for chr in char:
        if chr not in alphabet:
            alphabet.append(chr)

alphabet.sort()
alphabet


[',',
 '.',
 'C',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'z',
 'Ġ']

add the special tokens used by the model at the beginning of that vocabulary. In the case of GPT-2, the only special token is "<|endoftext|>"

In [20]:
vocab = ["<|endoftext|>"] + alphabet.copy()
vocab

['<|endoftext|>',
 ',',
 '.',
 'C',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'z',
 'Ġ']

4. split each word into individual characters, to be able to start training

In [62]:
splits = {word: [c for c in word] for word in word_freqs.keys()}
splits

{'This': ['T', 'h', 'i', 's'],
 'Ġis': ['Ġ', 'i', 's'],
 'Ġthe': ['Ġ', 't', 'h', 'e'],
 'ĠHugging': ['Ġ', 'H', 'u', 'g', 'g', 'i', 'n', 'g'],
 'ĠFace': ['Ġ', 'F', 'a', 'c', 'e'],
 'ĠCourse': ['Ġ', 'C', 'o', 'u', 'r', 's', 'e'],
 '.': ['.'],
 'Ġchapter': ['Ġ', 'c', 'h', 'a', 'p', 't', 'e', 'r'],
 'Ġabout': ['Ġ', 'a', 'b', 'o', 'u', 't'],
 'Ġtokenization': ['Ġ',
  't',
  'o',
  'k',
  'e',
  'n',
  'i',
  'z',
  'a',
  't',
  'i',
  'o',
  'n'],
 'Ġsection': ['Ġ', 's', 'e', 'c', 't', 'i', 'o', 'n'],
 'Ġshows': ['Ġ', 's', 'h', 'o', 'w', 's'],
 'Ġseveral': ['Ġ', 's', 'e', 'v', 'e', 'r', 'a', 'l'],
 'Ġtokenizer': ['Ġ', 't', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r'],
 'Ġalgorithms': ['Ġ', 'a', 'l', 'g', 'o', 'r', 'i', 't', 'h', 'm', 's'],
 'Hopefully': ['H', 'o', 'p', 'e', 'f', 'u', 'l', 'l', 'y'],
 ',': [','],
 'Ġyou': ['Ġ', 'y', 'o', 'u'],
 'Ġwill': ['Ġ', 'w', 'i', 'l', 'l'],
 'Ġbe': ['Ġ', 'b', 'e'],
 'Ġable': ['Ġ', 'a', 'b', 'l', 'e'],
 'Ġto': ['Ġ', 't', 'o'],
 'Ġunderstand': ['Ġ', 'u', 'n'

5. computes the frequency of each pair. We’ll need to use this at each step of the training

In [25]:
def compute_pair_freqs(splits):
    pair_freq = dict()
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i+1])
            if pair not in pair_freq:
                pair_freq[pair] = freq
            else:
                pair_freq[pair] += freq
    
    return pair_freq


In [27]:
pair_freqs = compute_pair_freqs(splits)
pair_freqs

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('T', 'h'): 3
('h', 'i'): 3
('i', 's'): 5
('Ġ', 'i'): 2
('Ġ', 't'): 7
('t', 'h'): 3


In [30]:
pair_freqs

{('T', 'h'): 3,
 ('h', 'i'): 3,
 ('i', 's'): 5,
 ('Ġ', 'i'): 2,
 ('Ġ', 't'): 7,
 ('t', 'h'): 3,
 ('h', 'e'): 2,
 ('Ġ', 'H'): 1,
 ('H', 'u'): 1,
 ('u', 'g'): 1,
 ('g', 'g'): 1,
 ('g', 'i'): 1,
 ('i', 'n'): 2,
 ('n', 'g'): 1,
 ('Ġ', 'F'): 1,
 ('F', 'a'): 1,
 ('a', 'c'): 1,
 ('c', 'e'): 1,
 ('Ġ', 'C'): 1,
 ('C', 'o'): 1,
 ('o', 'u'): 3,
 ('u', 'r'): 1,
 ('r', 's'): 2,
 ('s', 'e'): 3,
 ('Ġ', 'c'): 1,
 ('c', 'h'): 1,
 ('h', 'a'): 1,
 ('a', 'p'): 1,
 ('p', 't'): 1,
 ('t', 'e'): 2,
 ('e', 'r'): 5,
 ('Ġ', 'a'): 5,
 ('a', 'b'): 2,
 ('b', 'o'): 1,
 ('u', 't'): 1,
 ('t', 'o'): 4,
 ('o', 'k'): 3,
 ('k', 'e'): 3,
 ('e', 'n'): 4,
 ('n', 'i'): 2,
 ('i', 'z'): 2,
 ('z', 'a'): 1,
 ('a', 't'): 2,
 ('t', 'i'): 2,
 ('i', 'o'): 2,
 ('o', 'n'): 2,
 ('Ġ', 's'): 3,
 ('e', 'c'): 1,
 ('c', 't'): 1,
 ('s', 'h'): 1,
 ('h', 'o'): 2,
 ('o', 'w'): 2,
 ('w', 's'): 1,
 ('e', 'v'): 1,
 ('v', 'e'): 1,
 ('r', 'a'): 3,
 ('a', 'l'): 2,
 ('z', 'e'): 1,
 ('l', 'g'): 1,
 ('g', 'o'): 1,
 ('o', 'r'): 1,
 ('r', 'i'): 1,
 ('i', '

In [15]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('Ġ', 't') 7


So the first merge to learn is ('Ġ', 't') -> 'Ġt', and we add 'Ġt' to the vocabulary:

In [28]:
merges = {("Ġ", "t"): "Ġt"}
vocab.append("Ġt")

In [76]:
def merge_pair(a, b, splits):

    for words in splits.keys():
        split = splits[words]
        if len(split) == 1:
            continue
        
        i = 0
        while i < len(split)-1:
            if split[i] == a and split[i+1] == b:
                split = split[:i] + [a+b] + split[i+2:]
            else:
                i += 1
            
        splits[words] = split
    
    return splits        

TEST!

In [77]:
splits = merge_pair("Ġ", "t", splits)
print(splits["Ġtrained"])

['Ġt', 'r', 'a', 'in', 'e', 'd']


Now we have everything we need to loop until we have learned all the merges we want. Let’s aim for a vocab size of 50:

In [75]:
vocab_size = 50
while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
        
    splits = merge_pair (*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

vocab


['<|endoftext|>',
 ',',
 '.',
 'C',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'z',
 'Ġ',
 'Ġt',
 'er',
 'Ġa',
 'Ġto',
 'en',
 'Thi',
 'This',
 'ou',
 'se',
 'Ġtok',
 'Ġtoken',
 'nd',
 'Ġi',
 'Ġis',
 'Ġth',
 'Ġthe',
 'in',
 'Ġab',
 'Ġtokeni']

In [82]:
merges

{('Ġ', 't'): 'Ġt',
 ('e', 'r'): 'er',
 ('Ġ', 'a'): 'Ġa',
 ('Ġt', 'o'): 'Ġto',
 ('e', 'n'): 'en',
 ('T', 'hi'): 'Thi',
 ('Thi', 's'): 'This',
 ('o', 'u'): 'ou',
 ('s', 'e'): 'se',
 ('Ġto', 'k'): 'Ġtok',
 ('Ġtok', 'en'): 'Ġtoken',
 ('n', 'd'): 'nd',
 ('Ġ', 'i'): 'Ġi',
 ('Ġi', 's'): 'Ġis',
 ('Ġt', 'h'): 'Ġth',
 ('Ġth', 'e'): 'Ġthe',
 ('i', 'n'): 'in',
 ('Ġa', 'b'): 'Ġab',
 ('Ġtoken', 'i'): 'Ġtokeni'}

To tokenize a new text, we pre-tokenize it, split it, then apply all the merge rules learned:

In [99]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenize_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenize_text]
    print(f"{splits}")
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:  #KHEILI KHAFAN BOOD :D.
                                       # roo khodesh dare loop mizane 
                                       #moratrab avalin charachter ro ba badi
                                       #check mikone bebine
                                       #ke bebine toolanitarin kalameii ke bar asas
                                       #merge rule mitoone besaze chiye. YAY!
                                       #ALGORITHM KHOOBI BOOOOOD :D
                if split[i] == pair[0] and split[i+1] == pair[1]:
                    split = split[:i] + [merge] + split[i+2:]
                    print(f"{split= }")
                    print(i)
                else:
                    i += 1
            splits[idx] = split
    
    return sum(splits, [])


In [81]:
tokenize("This is not a token.")

['T', 'h', 'i', 's', 'Ġis', 'Ġ', 'n', 'o', 't', 'Ġa', 'Ġtoken', '.']

In [100]:
tokenize("This is is is.")

[['T', 'h', 'i', 's'], ['Ġ', 'i', 's'], ['Ġ', 'i', 's'], ['Ġ', 'i', 's'], ['.']]
split= ['Ġi', 's']
0
split= ['Ġi', 's']
0
split= ['Ġi', 's']
0
split= ['Ġis']
0
split= ['Ġis']
0
split= ['Ġis']
0


['T', 'h', 'i', 's', 'Ġis', 'Ġis', 'Ġis', '.']